<a href="https://colab.research.google.com/github/Sureshkumar0406/LLM_Fine_tuning_LORA_Quantization/blob/main/Fine_Tuning_LLM_Memorization_using_LORA_%26_Quantization_techniques.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import torch
print(torch.cuda.is_available())

True


In [44]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 89.6 gigabytes of available RAM

You are using a high-RAM runtime!


In [5]:
!nvcc --version  # Check CUDA version


nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2024 NVIDIA Corporation
Built on Thu_Jun__6_02:18:23_PDT_2024
Cuda compilation tools, release 12.5, V12.5.82
Build cuda_12.5.r12.5/compiler.34385749_0


In [6]:
!pip install torch --index-url https://download.pytorch.org/whl/cu121


Looking in indexes: https://download.pytorch.org/whl/cu121


In [7]:
#Install necessary libraries & its dependencies

!python -m pip install --upgrade pip -q
!pip install transformers==4.38.1 -q -U
!pip install bitsandbytes==0.42.0 -q -U
!pip install peft==0.8.2 -q -U
# !pip install flash-attn==2.5.5 -q -U
!pip install datasets==2.17.1 -q -U
!pip install scipy==1.12.0 -q -U
!pip install trl==0.7.11 -q -U
!pip install hf_transfer==0.1.5 -q -U
!pip install huggingface_hub==0.20.3 -q -U
!pip install wandb==0.16.3 -q -U

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 76.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 85.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 99.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sentence-transformers 3.4.1 requires transformers<5.0.0,>=4.41.0, but you have transformers 4.38.1 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 72.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 38.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 138.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 123.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 52.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 29.7 MB/s 

In [8]:
!pip install accelerate==0.27.2 -q -U

In [9]:
# Install & Save Packages to Google Drive to aovid re installing after kernel or session restarts


from google.colab import drive
drive.mount('/content/drive')

# Create a directory in Google Drive for storing installed packages
!mkdir -p /content/drive/MyDrive/colab_packages

# # Install necessary packages
# !pip install transformers datasets accelerate

# Save installed packages to Google Drive
!cp -r /usr/local/lib/python3.*/dist-packages /content/drive/MyDrive/colab_packages/


#  # Reload Packages Without Reinstalling (After Restart)


# from google.colab import drive
# drive.mount('/content/drive')

# # Link the saved packages from Google Drive to Colab's system path
# !cp -r /content/drive/MyDrive/colab_packages/dist-packages /usr/local/lib/python3.*/

# # Verify if packages are available
# !pip list | grep transformers




Mounted at /content/drive
^C


In [4]:
import os

# Define cache directory
CACHE_DIR = "/content/model_cache"

# Ensure the directory exists
os.makedirs(CACHE_DIR, exist_ok=True)

print(f"Cache directory set to: {CACHE_DIR}")


Cache directory set to: /content/model_cache


In [5]:
#Loading th eMistral model for Extractive Question Answering Task

model_id = "openchat/openchat_3.5"

In [13]:
# Install & Save Packages to Google Drive to aovid re installing after kernel or session restarts

from google.colab import drive
drive.mount('/content/drive')

# Create a directory in Google Drive for storing installed packages
!mkdir -p /content/drive/MyDrive/colab_packages

# Uninstall and reinstall transformers
!pip uninstall transformers -y
!pip install transformers

# Save installed packages to Google Drive
!cp -r /usr/local/lib/python3.*/dist-packages /content/drive/MyDrive/colab_packages/

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Found existing installation: transformers 4.38.1
Uninstalling transformers-4.38.1:
  Successfully uninstalled transformers-4.38.1
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 127.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 88.0 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.20.3
    Uninstalling huggingface-hub-0.20.3:
      Successfully uninstalled huggingface-hub-0.20.3
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.15.2
    Uninstalling tokenizers-0.15.2:
      Successfully uninstalled tokenizers-0.15.2


^C


In [11]:
!pip install --upgrade transformers

  Using cached transformers-4.48.3-py3-none-any.whl.metadata (44 kB)
  Using cached tokenizers-0.21.0-cp39-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.7 kB)
Using cached transformers-4.48.3-py3-none-any.whl (9.7 MB)
Using cached tokenizers-0.21.0-cp39-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (3.0 MB)
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.13.3
    Uninstalling tokenizers-0.13.3:
      Successfully uninstalled tokenizers-0.13.3
  Attempting uninstall: transformers
    Found existing installation: transformers 4.28.0
    Uninstalling transformers-4.28.0:
      Successfully uninstalled transformers-4.28.0


In [2]:
# Configure the Quantization to avoid memory issues

from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,  # Enables 4-bit quantization
    bnb_4bit_use_double_quant=True,  # Enables double quantization
    bnb_4bit_quant_type="nf4",  # Uses NormalFloat4 for quantization
    bnb_4bit_compute_dtype=torch.float16,
    load_in_8bit_fp32_cpu_offload=True
)

In [6]:
# Load the Model with  Quantization

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    # rope_scaling={"type": "linear", "factor": 2.0},
    device_map='auto',
    # trust_remote_code=False,
    torch_dtype=torch.float16,
    cache_dir = CACHE_DIR)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:1142: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:1142: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:410: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.5` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:410: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.5` -- this flag is only used in sample-based generation modes. You should se

In [9]:

tokenizer = AutoTokenizer.from_pretrained(model_id,use_fast=True,trust_remote_code=True)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [10]:

#Save tokenizer to a directory
tokenizer.save_pretrained("./saved_tokenizer")



# #ReLoad tokenizer from saved directory in case if the server or session got restarted
# tokenizer = AutoTokenizer.from_pretrained("./saved_tokenizer")

# print("Tokenizer loaded successfully!")



('./saved_tokenizer/tokenizer_config.json',
 './saved_tokenizer/special_tokens_map.json',
 './saved_tokenizer/tokenizer.model',
 './saved_tokenizer/added_tokens.json',
 './saved_tokenizer/tokenizer.json')

In [6]:
# Check there are no parameters overflowing onto cpu (meta).
for n, p in model.named_parameters():
    if p.device.type == "meta":
        print(f"{n} is on meta!")

In [7]:
print(model.config.max_position_embeddings)
print(model.config.eos_token_id)

8192
32000


In [8]:
import os
os.environ["TOKENIZERS_PARALLELISM"] = "true"


In [15]:
!pip install --upgrade datasets huggingface_hub

  Attempting uninstall: huggingface_hub
    Found existing installation: huggingface-hub 0.20.3
    Uninstalling huggingface-hub-0.20.3:
      Successfully uninstalled huggingface-hub-0.20.3
  Attempting uninstall: datasets
    Found existing installation: datasets 2.17.1
    Uninstalling datasets-2.17.1:
      Successfully uninstalled datasets-2.17.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sentence-transformers 3.4.1 requires transformers<5.0.0,>=4.41.0, but you have transformers 4.38.1 which is incompatible.


In [12]:
#Loading the dataset

from datasets import load_dataset

dataset="Trelis/touch-rugby-rules-memorisation"

data = load_dataset(dataset)



In [13]:
data['train'][0]

{'messages': '{"messages": [{"role": "user", "content": "What edition of the Touch Football rules was presented by Touch Football Australia at the Federation of International Touch Paris Convention in October 2019?"}, {"role": "assistant", "content": "The TFA 8th Edition Playing Rules were presented by Touch Football Australia at the Federation of International Touch Paris Convention in October 2019."}, {"role": "user", "content": "Who resolved to adopt the TFA 8th Edition Playing Rules as the 5th Edition FIT Playing Rules internationally?"}, {"role": "assistant", "content": "The FIT Board resolved to adopt the TFA 8th Edition Playing Rules as the 5th Edition FIT Playing Rules to be used across all levels of the game internationally."}, {"role": "user", "content": "Are National Touch Associations (NTAs) required to pay royalties to use the newly developed rules presented by Touch Football Australia?"}, {"role": "assistant", "content": "No, Touch Football Australia offered FIT and all N

In [11]:
# Extract text from the first row of 'test' in data
text = data['train'][0]['messages']

# Make sure to define or import the tokenizer
from transformers import AutoTokenizer

# Define model_id here
model_id = "openchat/openchat_3.5"

tokenizer = AutoTokenizer.from_pretrained(model_id, use_fast=True, trust_remote_code=True)

# Tokenize the text
tokens = tokenizer.encode(text, add_special_tokens=True)

# Decode back to text
decoded_text = tokenizer.decode(tokens)

# Print tokens and decoded text
print("Token IDs:", tokens)
print("Decoded Text:", decoded_text)

Token IDs: [1, 9830, 16167, 1264, 733, 6799, 9606, 1264, 345, 1838, 548, 345, 3789, 1264, 345, 3195, 12065, 302, 272, 25086, 13474, 5879, 403, 7567, 486, 25086, 13474, 6664, 438, 272, 22520, 302, 5440, 25086, 5465, 23717, 297, 4527, 28705, 28750, 28734, 28740, 28774, 1110, 881, 9830, 9606, 1264, 345, 489, 11143, 548, 345, 3789, 1264, 345, 1014, 320, 3120, 28705, 28783, 362, 20356, 6879, 288, 24759, 654, 7567, 486, 25086, 13474, 6664, 438, 272, 22520, 302, 5440, 25086, 5465, 23717, 297, 4527, 28705, 28750, 28734, 28740, 28774, 611, 881, 9830, 9606, 1264, 345, 1838, 548, 345, 3789, 1264, 345, 11447, 15813, 298, 8799, 272, 320, 3120, 28705, 28783, 362, 20356, 6879, 288, 24759, 390, 272, 28705, 28782, 362, 20356, 12630, 6879, 288, 24759, 17861, 578, 1110, 881, 9830, 9606, 1264, 345, 489, 11143, 548, 345, 3789, 1264, 345, 1014, 12630, 9217, 15813, 298, 8799, 272, 320, 3120, 28705, 28783, 362, 20356, 6879, 288, 24759, 390, 272, 28705, 28782, 362, 20356, 12630, 6879, 288, 24759, 298, 347, 130

In [14]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model and lists which parameters are trainable.
    """
    trainable_params = 0
    non_trainable_params = 0
    all_params = 0

    print("Trainable Parameters:")
    for name, param in model.named_parameters():
        all_params += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
            print(f"  {name}")
        else:
            non_trainable_params += param.numel()

    print("\nNon-Trainable Parameters:")
    for name, param in model.named_parameters():
        if not param.requires_grad:
            print(f"  {name}")

    print(
        f"\nSummary:\n  Trainable params: {trainable_params}\n  Non-Trainable params: {non_trainable_params}\n  All params: {all_params}\n  Trainable%: {100 * trainable_params / all_params}"
    )

In [19]:
!pip install --upgrade bitsandbytes


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 MB 94.7 MB/s eta 0:00:00
  Attempting uninstall: bitsandbytes
    Found existing installation: bitsandbytes 0.42.0
    Uninstalling bitsandbytes-0.42.0:
      Successfully uninstalled bitsandbytes-0.42.0


In [22]:
import os
os.environ['LD_LIBRARY_PATH'] = '/usr/local/cuda/lib64'

In [1]:
!python -m bitsandbytes

++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
++++++++++++++++++ BUG REPORT INFORMATION ++++++++++++++++++
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
++++++++++++++++++++++++++ OTHER +++++++++++++++++++++++++++
CUDA specs: CUDASpecs(highest_compute_capability=(8, 0), cuda_version_string='124', cuda_version_tuple=(12, 4))
PyTorch settings found: CUDA_VERSION=124, Highest Compute Capability: (8, 0).
To manually override the PyTorch CUDA version please see: https://github.com/TimDettmers/bitsandbytes/blob/main/docs/source/nonpytorchcuda.mdx
The directory listed in your path is found to be non-existent: /sys/fs/cgroup/memory.events /var/colab/cgroup/jupyter-children/memory.events
The directory listed in your path is found to be non-existent: //172.28.0.1
The directory listed in your path is found to be non-existent: 8013
The directory listed in your path is found to be non-existent: //colab.research.google.com/tun/m/cc48301118ce562b961b3c22d803539adc1e0c19/

In [15]:
# Set up LORA for the model fine tuning

from peft import prepare_model_for_kbit_training

model.gradient_checkpointing_enable()

from peft import LoraConfig, get_peft_model

peft_config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=[
        "q_proj",
        "k_proj",
        "v_proj",
        "o_proj",
        "gate_proj",
        "up_proj",
        "down_proj"
    ],
    lora_dropout=0.1,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, peft_config)

In [14]:
model

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): MistralForCausalLM(
      (model): MistralModel(
        (embed_tokens): Embedding(32002, 4096)
        (layers): ModuleList(
          (0-31): 32 x MistralDecoderLayer(
            (self_attn): MistralAttention(
              (q_proj): lora.Linear(
                (base_layer): Linear(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.1, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=8, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=8, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
              )
              (k_proj): lora.Linear(
                (base_layer): Linear(in_features

In [15]:
print_trainable_parameters(model)


Trainable Parameters:
  base_model.model.model.layers.0.self_attn.q_proj.lora_A.default.weight
  base_model.model.model.layers.0.self_attn.q_proj.lora_B.default.weight
  base_model.model.model.layers.0.self_attn.k_proj.lora_A.default.weight
  base_model.model.model.layers.0.self_attn.k_proj.lora_B.default.weight
  base_model.model.model.layers.0.self_attn.v_proj.lora_A.default.weight
  base_model.model.model.layers.0.self_attn.v_proj.lora_B.default.weight
  base_model.model.model.layers.0.self_attn.o_proj.lora_A.default.weight
  base_model.model.model.layers.0.self_attn.o_proj.lora_B.default.weight
  base_model.model.model.layers.0.mlp.gate_proj.lora_A.default.weight
  base_model.model.model.layers.0.mlp.gate_proj.lora_B.default.weight
  base_model.model.model.layers.0.mlp.up_proj.lora_A.default.weight
  base_model.model.model.layers.0.mlp.up_proj.lora_B.default.weight
  base_model.model.model.layers.0.mlp.down_proj.lora_A.default.weight
  base_model.model.model.layers.0.mlp.down_proj.

In [16]:
print(tokenizer)
print(tokenizer.vocab_size)

LlamaTokenizerFast(name_or_path='openchat/openchat_3.5', vocab_size=32000, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='left', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '<|end_of_turn|>', 'unk_token': '<unk>', 'additional_special_tokens': ['<|end_of_turn|>', '<|pad_0|>']}, clean_up_tokenization_spaces=False, added_tokens_decoder={
	0: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	32000: AddedToken("<|end_of_turn|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	32001: AddedToken("<|pad_0|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}
)
32000


In [17]:
#Test the chat template
messages=[
    { 'role': 'user', 'content': "write a quick sort algorithm in python."},
    { 'role': 'assistant', 'content': "here you are."},
    { 'role': 'user', 'content': "great."},
]

inputs = tokenizer.apply_chat_template(messages, tokenize=False)
print(inputs)

<s>GPT4 Correct User: write a quick sort algorithm in python.<|end_of_turn|>GPT4 Correct Assistant: here you are.<|end_of_turn|>GPT4 Correct User: great.<|end_of_turn|>


In [16]:
## Set up padding. Lets set the pad token to <pad>, if not <|pad|>, if not <unk> if <unk> is in the tokenizer OR set it to the EOS token.
if '<pad>' in tokenizer.get_vocab():
    print('<pad> token is in the tokenizer. Using <pad> for pad')

    tokenizer.pad_token = '<pad>'
elif '<|pad|>' in tokenizer.get_vocab():
    print('<|pad|> token is in the tokenizer. Using <|pad|> for pad')

    tokenizer.pad_token = '<|pad|>'
elif '<unk>' in tokenizer.get_vocab():
    print('<unk> token is in the tokenizer. Using unk for pad')

    tokenizer.pad_token = '<unk>'
else:
    print(f'Using EOS token, {tokenizer.eos_token}, for padding. WARNING, this may not be ideal for chat fine-tuning models.')
    tokenizer.pad_token = tokenizer.eos_token

<unk> token is in the tokenizer. Using unk for pad


In [17]:
# Update pad token id in model and its config
model.pad_token_id = tokenizer.pad_token_id
model.config.pad_token_id = tokenizer.pad_token_id

# Check if they are equal
assert model.pad_token_id == tokenizer.pad_token_id, "The model's pad token ID does not match the tokenizer's pad token ID!"

# Print the pad token ids
print('Tokenizer pad token ID:', tokenizer.pad_token_id)
print('Model pad token ID:', model.pad_token_id)
print('Model config pad token ID:', model.config.pad_token_id)
print('Number of tokens now in tokenizer:', tokenizer.vocab_size)

Tokenizer pad token ID: 0
Model pad token ID: 0
Model config pad token ID: 0
Number of tokens now in tokenizer: 32000


In [18]:
print("Special tokens map:", tokenizer.special_tokens_map)
# print("All special tokens:", tokenizer.all_special_tokens)

Special tokens map: {'bos_token': '<s>', 'eos_token': '<|end_of_turn|>', 'unk_token': '<unk>', 'pad_token': '<unk>', 'additional_special_tokens': ['<|end_of_turn|>', '<|pad_0|>']}


In [19]:
#Define streaming fucntion & evaluate it

from transformers import TextStreamer
from peft import PeftModel
import torch
import gc  # import Python's garbage collection module

# Define a stream
def stream(user_prompt, model_type, tokenizer, checkpoint=''):

    if model_type == 'base':
        eval_model = model
    elif model_type == 'fine-tuned':
        eval_model = PeftModel.from_pretrained(model, checkpoint)  # Assuming PeftModel is the intended class
        eval_model = eval_model.to("cuda")

        for n, p in eval_model.named_parameters():
            if p.device.type == "cpu":
                print(f"{n} is on cpu!")

    else:
        print('You must set the model_type to base or fine-tuned')
        exit()  # or raise an exception

    # print(f'Proceeding to inference with peft adapters from {checkpoint}')

    eval_model.config.use_cache = True

    messages=[
        { 'role': 'user', 'content': f"{user_prompt.strip()}"},
    ]

    inputs = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

    inputs = tokenizer([inputs], return_tensors="pt", add_special_tokens=False).to("cuda")

    if "token_type_ids" in inputs:
        del inputs["token_type_ids"]

    streamer = TextStreamer(tokenizer)

    print(f'eval_model is on: {next(eval_model.parameters()).device}')  # Debug line
    print(f'input_ids are on: {inputs["input_ids"].device}')  # Debug line

    # Despite returning the usual output, the streamer will also print the generated text to stdout.
    # _ = eval_model.generate(**inputs, streamer=streamer)
    _ = eval_model.generate(**inputs, streamer=streamer, max_new_tokens=100, do_sample=False, pad_token_id=tokenizer.pad_token_id, eos_token_id=tokenizer.eos_token_id,num_beams=1 )

    # Clear GPU cache and run garbage collection
    torch.cuda.empty_cache()  # Clear GPU cache
    gc.collect()  # Run garbage collection


def evaluation(model_type, tokenizer, checkpoint=''):
    questions = [
        "In the context of Touch Rugby International Playing Rules 2020, what is the purpose of the Dead Ball Line?", #copied from the test data set to ensure training is working
        "How many players are on the field on each team in touch rugby?",
        "In touch rugby, does a forward pass result in a roll ball, a scrum, or something else?",
        "In touch rugby, how many metres must the defending team retreat after a touch?",
        "In touch rugby, how many substitutions are allowed during a game?",
        "In touch rugby, how long is half time?",
        "In touch rugby, how does the game commence?",
        "In touch rugby, how many metres must defenders retreat when there is a penalty? Is the same as after a touch is made?",
        "In touch rugby, how many touches is a team entitled to prior to a change in possession?",
        "In touch rugby, what happens if a player makes a pass after a touch has been made?",
        "In touch rugby, how many points is a try worth?"
    ]

    answers = [
        "The Dead Ball Line marks the end boundaries of the field of play and indicates when the ball is out of play.",
        "6 players.",
        "Penalty.",
        "7 metres.",
        "There is no limit.",
        "5 minutes.",
        "The game begins with a tap on the halfway line.",
        "10 metres.",
        "Possession changes on the sixth (6th) touch.",
        "The defending team gains possession and a penalty.",
        "1 point."
    ]

    for question, answer in zip(questions, answers):
        stream(question, model_type, tokenizer, checkpoint)
        print("Correct Answer:", answer)
        print('\n\n')

In [20]:
print(model.generation_config)


GenerationConfig {
  "bos_token_id": 1,
  "eos_token_id": 32000,
  "max_length": 8192,
  "pad_token_id": 0,
  "temperature": 0.5
}



In [21]:
#Evaluate the base pre-trained model

evaluation("base", tokenizer)


eval_model is on: cuda:0
input_ids are on: cuda:0


/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:410: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.5` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


<s> GPT4 Correct User: In the context of Touch Rugby International Playing Rules 2020, what is the purpose of the Dead Ball Line?<|end_of_turn|> GPT4 Correct Assistant: The Dead Ball Line in Touch Rugby International Playing Rules 2020 serves as a boundary line on the field. When a ball goes beyond this line, it is considered out of play. The team that last touched the ball before it went out of play will be awarded a scrum or a line-out, depending on the situation. The Dead Ball Line helps to maintain the flow of the game and ensures that the ball remains in play within the designated area.<|end_of_turn|>
Correct Answer: The Dead Ball Line marks the end boundaries of the field of play and indicates when the ball is out of play.



eval_model is on: cuda:0
input_ids are on: cuda:0
<s> GPT4 Correct User: How many players are on the field on each team in touch rugby?<|end_of_turn|> GPT4 Correct Assistant: In 

/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:410: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.5` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


touch rugby, each team has 12 players on the field at any given time. Therefore, the answer is 12.<|end_of_turn|>
Correct Answer: 6 players.



eval_model is on: cuda:0
input_ids are on: cuda:0
<s> GPT4 Correct User: In touch rugby, does a forward pass result in a roll ball, a scrum, or something else?<|end_of_turn|> GPT4 Correct Assistant: In touch rugby, a forward pass results in a roll ball.<|end_of_turn|>
Correct Answer: Penalty.



eval_model is on: cuda:0
input_ids are on: cuda:0
<s> GPT4 Correct User: In touch rugby, how many metres must the defending team retreat after a touch?<|end_of_turn|> GPT4 Correct Assistant: In touch rugby, the defending team must retreat 5 metres after a touch. Therefore, the answer is 5.<|end_of_turn|>
Correct Answer: 7 metres.



eval_model is on: cuda:0
input_ids are on: cuda:0
<s> GPT4 Correct User: In touch rugby, how many substitutions are allowed during a game?<|end_of_turn|> GPT4 Correct Assistant: In touch rugby, there is no limit to the numbe

In [2]:
!pip install transformers==4.31.0
!pip install trl==0.7.11

  Using cached transformers-4.31.0-py3-none-any.whl.metadata (116 kB)
  Using cached tokenizers-0.13.3-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.7 kB)
Using cached transformers-4.31.0-py3-none-any.whl (7.4 MB)
Using cached tokenizers-0.13.3-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (7.8 MB)
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.21.0
    Uninstalling tokenizers-0.21.0:
      Successfully uninstalled tokenizers-0.21.0
  Attempting uninstall: transformers
    Found existing installation: transformers 4.48.3
    Uninstalling transformers-4.48.3:
      Successfully uninstalled transformers-4.48.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sentence-transformers 3.4.1 requires transformers<5.0.0,>=4.41.0, but you have transformers 4.31.0 which is incompatible.


In [22]:
model_name = model_id.split("/")[-1]
dataset_name = dataset.split("/")[-1]

epochs=1
context_length = 512
grad_accum=1
batch_size=1
fine_tune_tag='touch-rugby-rules'
save_dir = f'./results/{model_name}_{dataset_name}_{epochs}_epochs_{context_length}_length-{fine_tune_tag}'
print(save_dir)

./results/openchat_3.5_touch-rugby-rules-memorisation_1_epochs_512_length-touch-rugby-rules


In [2]:
!pip install huggingface_hub==0.25.2

  Attempting uninstall: huggingface_hub
    Found existing installation: huggingface-hub 0.28.1
    Uninstalling huggingface-hub-0.28.1:
      Successfully uninstalled huggingface-hub-0.28.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sentence-transformers 3.4.1 requires transformers<5.0.0,>=4.41.0, but you have transformers 4.38.1 which is incompatible.


In [4]:
import transformers
print(transformers.__version__)  # Should be 4.38.1 or higher


4.31.0


In [24]:
from transformers import Trainer
from trl import SFTTrainer
import transformers


/usr/local/lib/python3.11/dist-packages/diffusers/utils/outputs.py:63: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  torch.utils._pytree._register_pytree_node(
/usr/local/lib/python3.11/dist-packages/diffusers/utils/outputs.py:63: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  torch.utils._pytree._register_pytree_node(


In [25]:
# set up trianing argumaents & trian the model

trainer = SFTTrainer(
    # peft_config=peft_config, #comment out if passing a peft model directly as 'model'
    dataset_text_field="messages",
    max_seq_length=context_length,
    tokenizer=tokenizer,
    model=model,
    train_dataset=data["train"],
    eval_dataset=data["test"],
    args=transformers.TrainingArguments(
        # max_steps=1, # comment this out after the first time you run. This is for testing!
        save_steps=10, ### MAKE SURE TO CHECK THIS VALUE IS GOOD FOR YOUR RUN!
        logging_steps=1,
        num_train_epochs=epochs,
        output_dir=save_dir,
        evaluation_strategy="steps",
        do_eval=True,
        eval_steps=0.2,
        per_device_eval_batch_size=batch_size,
        per_device_train_batch_size=batch_size,
        gradient_accumulation_steps=grad_accum,
        log_level="debug",
        bf16=True,
        max_grad_norm=0.3,
        lr_scheduler_type="cosine",
        hub_private_repo=True,
        warmup_ratio=0.03, # optional, may help stability at the start of training. Not required for simple fine-tunes.
        optim="adamw_torch", #comment out for LoRA +
        learning_rate=1e-4, #comment out for LoRA +
    ),
    #callbacks=[logging_callback],  # Add custom callback here
    # optimizers=(optimizer, None),  # Comment in for LoRA+
    # neftune_noise_alpha=5 # Add in noise to embeddings to improve performance!
)

Map:   0%|          | 0/60 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/trl/trainer/sft_trainer.py:294: UserWarning: You passed a tokenizer with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a model in half-precision. You might consider adding `tokenizer.padding_side = 'right'` to your code.
  warnings.warn(
Using auto half precision backend


In [27]:
trainer.train()

Currently training with a batch size of: 1
***** Running training *****
  Num examples = 303
  Num Epochs = 1
  Instantaneous batch size per device = 1
  Total train batch size (w. parallel, distributed & accumulation) = 1
  Gradient Accumulation steps = 1
  Total optimization steps = 303
  Number of trainable parameters = 20,971,520
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"
wandb: Currently logged in as: sureshkumar-sekar86 (sureshkumar-sekar86-virtusa). Use `wandb login --relogin` to force relogin


`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...


Step,Training Loss,Validation Loss
61,0.952400,0.838154
122,0.608200,0.701534
183,0.337900,0.638882
244,0.480100,0.601945


Saving model checkpoint to ./results/openchat_3.5_touch-rugby-rules-memorisation_1_epochs_512_length-touch-rugby-rules/tmp-checkpoint-10
/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:1142: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--openchat--openchat_3.5/snapshots/0fc98e324280bc4bf5d2c30ecf7b97b84fb8a19b/config.json
Model config MistralConfig {
  "_name_or_path": "imone/Mistral_7B_with_EOT_token",
  "architectures": [
    "MistralForCausalLM"
  ],
  "attention_dropout": 0.0,
  "bos_token_id": 1,
  "eos_token_id": 32000,
  "hidden_act": "silu",
  "hidden_size": 4096,
  "initializer_range": 0.02,
  "intermediate_size": 14336,
  "max_position_embeddings": 8192,
  "model_type": "mistral",
  "num_attention_heads": 32

TrainOutput(global_step=303, training_loss=0.6537407050923546, metrics={'train_runtime': 210.5409, 'train_samples_per_second': 1.439, 'train_steps_per_second': 1.439, 'total_flos': 4530759296581632.0, 'train_loss': 0.6537407050923546, 'epoch': 1.0})

In [32]:
#Evaluate the model post training

evaluation("base", tokenizer)

eval_model is on: cuda:0
input_ids are on: cuda:0
<s> GPT4 Correct User: In the context of Touch Rugby International Playing Rules 2020, what is the purpose of the Dead Ball Line?<|end_of_turn|> GPT4 Correct Assistant: 

/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:410: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.5` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/utils/checkpoint.py:87: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


The Dead Ball Line is the line where the ball is considered out of play. It is located at the end boundaries of the Field of Play.<|end_of_turn|>
Correct Answer: The Dead Ball Line marks the end boundaries of the field of play and indicates when the ball is out of play.



eval_model is on: cuda:0
input_ids are on: cuda:0
<s> GPT4 Correct User: How many players are on the field on each team in touch rugby?<|end_of_turn|> GPT4 Correct Assistant: Each team has six players on the field at any given time in touch rugby.<|end_of_turn|>
Correct Answer: 6 players.



eval_model is on: cuda:0
input_ids are on: cuda:0
<s> GPT4 Correct User: In touch rugby, does a forward pass result in a roll ball, a scrum, or something else?<|end_of_turn|> GPT4 Correct Assistant: A forward pass results in a roll ball in touch rugby.<|end_of_turn|>
Correct Answer: Penalty.



eval_model is on: cuda:0
input_ids are on: cuda:0
<s> GPT4 Correct User: In touch rugby, how many metres must the defending team retreat 